In [3]:
import pandas as pd
import datetime
from matplotlib import pyplot as plt
from dateutil import parser
from dateutil.relativedelta import *
from prophet import Prophet
import numpy as np
import seaborn as sn

In [4]:
covidData = pd.read_csv("E:/MPS Data Science/Summer 2022/Data Challenge/Datasets/July 16th, 22/owid_covid-19-data_latest_covid-19-world-cases-deaths-testing.csv")
covidData = covidData[['iso_code','continent','location','total_cases','total_deaths','new_cases', 'new_deaths','life_expectancy','median_age','date']]
display(covidData)

,iso_code,continent,location,total_cases,total_deaths,new_cases,new_deaths,life_expectancy,median_age,date
0,AFG,Asia,Afghanistan,5.0,NaN,5.0,NaN,64.83,18.6,2020-02-24
1,AFG,Asia,Afghanistan,5.0,NaN,0.0,NaN,64.83,18.6,2020-02-25
2,AFG,Asia,Afghanistan,5.0,NaN,0.0,NaN,64.83,18.6,2020-02-26
3,AFG,Asia,Afghanistan,5.0,NaN,0.0,NaN,64.83,18.6,2020-02-27
4,AFG,Asia,Afghanistan,5.0,NaN,0.0,NaN,64.83,18.6,2020-02-28
...,...,...,...,...,...,...,...,...,...,...
201494,ZWE,Africa,Zimbabwe,255953.0,5565.0,14.0,1.0,61.49,19.6,2022-07-11
201495,ZWE,Africa,Zimbabwe,255981.0,5565.0,28.0,0.0,61.49,19.6,2022-07-12
201496,ZWE,Africa,Zimbabwe,255981.0,5565.0,0.0,0.0,61.49,19.6,2022-07-13
201497,ZWE,Africa,Zimbabwe,256047.0,5566.0,66.0,1.0,61.49,19.6,2022-07-14


In [5]:
covidData['date']= pd.to_datetime(covidData['date'])
print(covidData.dtypes)

iso_code                   object
continent                  object
location                   object
total_cases               float64
total_deaths              float64
new_cases                 float64
new_deaths                float64
life_expectancy           float64
median_age                float64
date               datetime64[ns]
dtype: object


In [6]:
covidData = covidData.bfill() #Next Observation Carried Backward
display(covidData)

,iso_code,continent,location,total_cases,total_deaths,new_cases,new_deaths,life_expectancy,median_age,date
0,AFG,Asia,Afghanistan,5.0,1.0,5.0,1.0,64.83,18.6,2020-02-24
1,AFG,Asia,Afghanistan,5.0,1.0,0.0,1.0,64.83,18.6,2020-02-25
2,AFG,Asia,Afghanistan,5.0,1.0,0.0,1.0,64.83,18.6,2020-02-26
3,AFG,Asia,Afghanistan,5.0,1.0,0.0,1.0,64.83,18.6,2020-02-27
4,AFG,Asia,Afghanistan,5.0,1.0,0.0,1.0,64.83,18.6,2020-02-28
...,...,...,...,...,...,...,...,...,...,...
201494,ZWE,Africa,Zimbabwe,255953.0,5565.0,14.0,1.0,61.49,19.6,2022-07-11
201495,ZWE,Africa,Zimbabwe,255981.0,5565.0,28.0,0.0,61.49,19.6,2022-07-12
201496,ZWE,Africa,Zimbabwe,255981.0,5565.0,0.0,0.0,61.49,19.6,2022-07-13
201497,ZWE,Africa,Zimbabwe,256047.0,5566.0,66.0,1.0,61.49,19.6,2022-07-14


In [7]:
country = input("Enter a country\n")
covidCountry = covidData[(covidData.location == country)]
display(covidCountry)

Enter a country
India


,iso_code,continent,location,total_cases,total_deaths,new_cases,new_deaths,life_expectancy,median_age,date
83176,IND,Asia,India,1.0,1.0,1.0,1.0,69.66,28.2,2020-01-30
83177,IND,Asia,India,1.0,1.0,0.0,1.0,69.66,28.2,2020-01-31
83178,IND,Asia,India,1.0,1.0,0.0,1.0,69.66,28.2,2020-02-01
83179,IND,Asia,India,2.0,1.0,1.0,1.0,69.66,28.2,2020-02-02
83180,IND,Asia,India,3.0,1.0,1.0,1.0,69.66,28.2,2020-02-03
...,...,...,...,...,...,...,...,...,...,...
84069,IND,Asia,India,43652944.0,525474.0,13615.0,20.0,69.66,28.2,2022-07-11
84070,IND,Asia,India,43669850.0,525519.0,16906.0,45.0,69.66,28.2,2022-07-12
84071,IND,Asia,India,43689989.0,525557.0,20139.0,38.0,69.66,28.2,2022-07-13
84072,IND,Asia,India,43710027.0,525604.0,20038.0,47.0,69.66,28.2,2022-07-14


In [8]:
day = input("Enter a date in YYYY-MM-DD format\n")
endPeriod = pd.to_datetime(day)
startPeriod = endPeriod - relativedelta(months = 3)
startPeriod = pd.to_datetime(startPeriod.date())
endPeriod = pd.to_datetime(endPeriod.date())

Enter a date in YYYY-MM-DD format
2021-01-10


In [9]:
predictingDays = int(input("Enter number of days to predict: 7/14/21\n"))
testEndPeriod = endPeriod + relativedelta(days = predictingDays)
testStartPeriod = endPeriod
testEndPeriod = pd.to_datetime(testEndPeriod.date())

Enter number of days to predict: 7/14/21
14


In [10]:
slicingCovidCountry = covidCountry[(covidCountry['date'] > startPeriod) & (covidCountry['date'] <= endPeriod)]
testData  = covidCountry[(covidCountry['date'] > testStartPeriod) & (covidCountry['date'] <= testEndPeriod)]

In [11]:
datesCases = slicingCovidCountry.groupby('date').sum()['new_cases'].reset_index()

In [12]:
datesCases.columns = ['ds','y']
datesCases['ds'] = pd.to_datetime(datesCases['ds'])

In [17]:
m = Prophet(changepoint_prior_scale=0.7, growth = 'linear', holidays_prior_scale = 0,interval_width=0.95, daily_seasonality = True)
m.fit(datesCases)
futureDates = m.make_future_dataframe(periods=predictingDays)

04:08:38 - cmdstanpy - INFO - Chain [1] start processing
04:08:43 - cmdstanpy - INFO - Chain [1] done processing
04:08:43 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 3221225657 


RuntimeError: Error during optimization: 

In [ ]:
forecast = m.predict(futureDates)
forecasted = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecasted.columns = ['date', 'Forecasted_Value', 'Lower_Limit', 'Upper_Limit']
forecastedCovid = pd.merge(forecasted, testData, how="right", on="date")

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(forecastedCovid['date'],forecastedCovid['Forecasted_Value'], color = 'Blue', label = 'forecast')
plt.plot(forecastedCovid['date'],forecastedCovid['new_cases'], color = 'Red', label = 'actual')
plt.legend()

In [ ]:
datesDeaths = slicingCovidCountry.groupby('date').sum()['new_deaths'].reset_index()
datesDeaths.columns = ['ds','y']
datesDeaths['ds'] = pd.to_datetime(datesDeaths['ds'])
datesDeaths.tail(7)

In [ ]:
m = Prophet(changepoint_prior_scale=0.25, growth = 'linear', holidays_prior_scale=0, interval_width=0.95, daily_seasonality = True)
m.fit(datesDeaths)
futureDates = m.make_future_dataframe(periods=predictingDays)


In [ ]:
forecast = m.predict(futureDates)
forecasted = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
forecasted.columns = ['date', 'Forecasted Value', 'Lower Limit', 'Upper Limit']
forecastedCovid = pd.merge(forecasted, testData, how="right", on="date")

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(forecastedCovid['date'],forecastedCovid['Forecasted Value'], color = 'Blue', label = 'forecast')
plt.plot(forecastedCovid['date'],forecastedCovid['new_deaths'], color = 'Red', label = 'actual')
plt.legend()